# Stoichiometry Screening: Getting the Ratios Right

You've mastered chemical filters and compositional screening - now it's time to tackle a more focused challenge: **finding the right atomic ratios**. Once you know which elements to combine, how do you determine the optimal stoichiometry?

This tutorial will teach you to systematically explore stoichiometric spaces and identify chemically viable atomic ratios using SMACT.

## What You'll Learn

1. **Distinguish compositional from stoichiometric screening**
2. **Generate systematic stoichiometric spaces** for element combinations
3. **Apply chemical validation rules** to atomic ratios
4. **Screen specific structure types** (binary, perovskite, quaternary)
5. **Use ICSD occurrence data** to guide predictions
6. **Visualise stoichiometric validity** with interactive plots

## The Foundation: Previous Learning

This builds directly on your previous knowledge:
- **Chemical Filters**: Provide the validation rules we'll apply
- **Compositional Screening**: Identified promising element combinations
- **Stoichiometry Screening**: Now finds optimal atomic ratios

Let's begin exploring the systematic world of atomic ratios!

In [ ]:
# Import Libraries

import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# Materials science libraries
from pymatgen.core import Composition

# SMACT for stoichiometric screening
import smact
from smact import Element, Species
from smact.screening import smact_validity
from smact.utils.oxidation import ICSD24OxStatesFilter

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")

print("Libraries imported successfully!")
print("Ready to begin stoichiometry screening")

## Understanding the Difference: Compositional vs Stoichiometric Screening

Let's clarify this important distinction with concrete examples:

**Compositional Screening** asks: *"Should I combine Cu, Ti, and O?"*
- Explores which elements to use together
- Maps entire chemical systems like Cu-Ti-O
- Answers: "Yes, Cu-Ti-O is a viable system"

**Stoichiometry Screening** asks: *"Given Cu, Ti, and O, should I make CuTiO₃, Cu₂TiO₄, or CuTi₂O₅?"*
- Explores different atomic ratios within known systems
- Applies chemical rules to specific stoichiometries
- Answers: "CuTiO₃ is chemically valid, but Cu₅TiO₁₃ is not"

Both are essential for comprehensive materials discovery!

In [ ]:
# Step 1: Binary Stoichiometry Screening

def screen_binary_stoichiometries(element_A, element_B, max_stoich=5):
    """
    Screen all possible stoichiometries for a binary system A_x B_y.
    
    Args:
        element_A, element_B (str): Element symbols
        max_stoich (int): Maximum stoichiometric coefficient
        
    Returns:
        pandas.DataFrame: Results with compositions and validity
    """
    results = []
    
    # Test all combinations of stoichiometric coefficients
    for x in range(1, max_stoich + 1):
        for y in range(1, max_stoich + 1):
            # Create composition
            comp = Composition({element_A: x, element_B: y}).reduced_composition
            
            # Test chemical validity
            is_valid = smact_validity(comp)
            
            results.append({
                'stoichiometry': f"{element_A}{x}{element_B}{y}",
                'composition': comp,
                'formula': comp.reduced_formula,
                'valid': is_valid,
                'coeff_A': x,
                'coeff_B': y
            })
    
    # Create DataFrame and remove duplicates
    df = pd.DataFrame(results)
    df = df.drop_duplicates('formula').reset_index(drop=True)
    
    return df

# Example: Screen sodium chloride stoichiometries
print("Screening Na-Cl binary stoichiometries...")
nacl_results = screen_binary_stoichiometries("Na", "Cl", max_stoich=4)

print(f"\nFound {len(nacl_results)} unique stoichiometries:")
print(nacl_results[['formula', 'valid']].head(8))

# Count valid vs invalid
valid_count = nacl_results['valid'].sum()
total_count = len(nacl_results)
print(f"\nValid stoichiometries: {valid_count}/{total_count} ({valid_count/total_count*100:.1f}%)")

In [ ]:
# Step 2: Visualise Binary Stoichiometric Space

def plot_stoichiometric_grid(df, element_A, element_B, max_display=5):
    \"\"\"\n    Create a grid visualisation of binary stoichiometric validity.\n    \n    Args:\n        df (DataFrame): Results from binary screening\n        element_A, element_B (str): Element symbols\n        max_display (int): Maximum coefficient to display\n    \"\"\"\n    # Create grid matrix\n    grid = np.zeros((max_display, max_display))\n    \n    for _, row in df.iterrows():\n        if row['coeff_A'] <= max_display and row['coeff_B'] <= max_display:\n            x_idx = row['coeff_A'] - 1\n            y_idx = row['coeff_B'] - 1\n            grid[x_idx, y_idx] = 1 if row['valid'] else -1\n    \n    # Create plot\n    fig, ax = plt.subplots(figsize=(8, 6))\n    \n    # Plot grid with colour coding\n    im = ax.imshow(grid, cmap='RdYlGn', aspect='equal', origin='lower', \n                   extent=[0.5, max_display+0.5, 0.5, max_display+0.5])\n    \n    # Add text annotations\n    for i in range(max_display):\n        for j in range(max_display):\n            value = grid[i, j]\n            if value == 1:\n                text = \"Valid\"\n                colour = \"black\"\n            elif value == -1:\n                text = \"Invalid\"\n                colour = \"black\"\n            else:\n                text = \"\"\n                colour = \"grey\"\n            \n            ax.text(j+1, i+1, text, ha='center', va='center', \n                   color=colour, fontsize=10, weight='bold')\n    \n    # Formatting\n    ax.set_xlabel(f\"{element_B} stoichiometric coefficient\")\n    ax.set_ylabel(f\"{element_A} stoichiometric coefficient\")\n    ax.set_title(f\"Stoichiometric Validity: {element_A}-{element_B} System\")\n    \n    # Set ticks\n    ax.set_xticks(range(1, max_display+1))\n    ax.set_yticks(range(1, max_display+1))\n    \n    plt.tight_layout()\n    plt.show()\n\n# Plot the Na-Cl results\nplot_stoichiometric_grid(nacl_results, \"Na\", \"Cl\", max_display=4)\n\nprint(\"\\nGrid interpretation:\")\nprint(\"- Green (Valid): Stoichiometry passes SMACT chemical rules\")\nprint(\"- Red (Invalid): Stoichiometry violates charge neutrality or electronegativity rules\")\nprint(\"- Only NaCl (1:1 ratio) is chemically valid for this system\")"

In [ ]:
# Step 2.1: More Complex Binary Examples\n\n# Let's explore more interesting binary systems to demonstrate different patterns\nbinary_systems = [\n    (\"Mg\", \"O\"),   # Simple oxide - expect MgO\n    (\"Al\", \"O\"),   # Different stoichiometry - expect Al2O3\n    (\"Cu\", \"O\"),   # Multiple valid ratios - Cu2O and CuO\n    (\"Fe\", \"S\")    # Transition metal sulfide - FeS, Fe2S3, etc.\n]\n\nprint(\"Exploring diverse binary stoichiometric systems...\\n\")\n\n# Create a 2x2 subplot grid for multiple systems\nfig, axes = plt.subplots(2, 2, figsize=(15, 12))\naxes = axes.flatten()\n\nfor idx, (elem_A, elem_B) in enumerate(binary_systems):\n    # Screen the binary system\n    results = screen_binary_stoichiometries(elem_A, elem_B, max_stoich=5)\n    valid_results = results[results['valid']]\n    \n    print(f\"{elem_A}-{elem_B} System Analysis:\")\n    print(f\"  Valid stoichiometries: {len(valid_results)}/{len(results)}\")\n    if len(valid_results) > 0:\n        print(f\"  Valid formulas: {', '.join(valid_results['formula'].tolist())}\")\n    else:\n        print(f\"  No valid stoichiometries found\")\n    print()\n    \n    # Create grid for this system\n    max_display = 5\n    grid = np.zeros((max_display, max_display))\n    \n    for _, row in results.iterrows():\n        if row['coeff_A'] <= max_display and row['coeff_B'] <= max_display:\n            x_idx = row['coeff_A'] - 1\n            y_idx = row['coeff_B'] - 1\n            grid[x_idx, y_idx] = 1 if row['valid'] else -1\n    \n    # Plot on subplot\n    ax = axes[idx]\n    im = ax.imshow(grid, cmap='RdYlGn', aspect='equal', origin='lower',\n                   extent=[0.5, max_display+0.5, 0.5, max_display+0.5])\n    \n    # Add annotations\n    for i in range(max_display):\n        for j in range(max_display):\n            value = grid[i, j]\n            if value == 1:\n                text = \"V\"  # Valid (abbreviated for space)\n                colour = \"black\"\n            elif value == -1:\n                text = \"X\"  # Invalid\n                colour = \"black\"\n            else:\n                text = \"\"\n                colour = \"grey\"\n            \n            ax.text(j+1, i+1, text, ha='center', va='center',\n                   color=colour, fontsize=12, weight='bold')\n    \n    # Format subplot\n    ax.set_xlabel(f\"{elem_B} coefficient\")\n    ax.set_ylabel(f\"{elem_A} coefficient\")\n    ax.set_title(f\"{elem_A}-{elem_B} System\")\n    ax.set_xticks(range(1, max_display+1))\n    ax.set_yticks(range(1, max_display+1))\n\nplt.tight_layout()\nplt.show()\n\nprint(\"Key observations:\")\nprint(\"- Mg-O: Simple 2:1 ratio (MgO) - classic ionic compound\")\nprint(\"- Al-O: 2:3 ratio (Al₂O₃) - higher charge cation requires different stoichiometry\")\nprint(\"- Cu-O: Multiple valid ratios (Cu₂O, CuO) - variable oxidation states\")\nprint(\"- Fe-S: Complex pattern - transition metals show diverse stoichiometries\")"

In [ ]:
# Step 3: Ternary Perovskite Screening (ABX₃)

def screen_perovskite_stoichiometries(A_elements, B_elements, X_elements):
    """
    Screen ABX₃ perovskite stoichiometries systematically.
    
    Args:
        A_elements, B_elements, X_elements (list): Element lists for each site
        
    Returns:
        pandas.DataFrame: Perovskite screening results
    """
    results = []
    
    # Test all combinations of A, B, X elements
    for A in A_elements:
        for B in B_elements:
            for X in X_elements:
                # Create ABX₃ composition
                comp = Composition({A: 1, B: 1, X: 3})
                
                # Test validity
                is_valid = smact_validity(comp)
                
                results.append({
                    'A_element': A,
                    'B_element': B, 
                    'X_element': X,
                    'formula': comp.reduced_formula,
                    'composition': comp,
                    'valid': is_valid
                })
    
    return pd.DataFrame(results)

# Define element sets for perovskite screening
A_site = ["Ca", "Sr", "Ba"]  # Large cations
B_site = ["Ti", "Zr", "Sn"]  # Smaller cations
X_site = ["O"]               # Anions

print("Screening ABX₃ perovskite stoichiometries...")
perovskite_results = screen_perovskite_stoichiometries(A_site, B_site, X_site)

print(f"\nPerovskite screening results:")
print(perovskite_results[['formula', 'valid']])

# Analyse results
valid_perovskites = perovskite_results[perovskite_results['valid']]
print(f"\nValid perovskite compositions: {len(valid_perovskites)}/{len(perovskite_results)}")

if len(valid_perovskites) > 0:
    print("\nChemically valid perovskites found:")
    for _, row in valid_perovskites.iterrows():
        print(f"  {row['formula']} ({row['A_element']}-{row['B_element']}-{row['X_element']})")

In [ ]:
# Step 3.1: Advanced Ternary Systems - Beyond Perovskites\n\ndef screen_ternary_ratios(elements, target_ratios, max_mult=2):\n    \"\"\"\n    Screen specific ternary stoichiometric ratios.\n    \n    Args:\n        elements (list): Three elements to screen\n        target_ratios (list): List of ratio tuples to test\n        max_mult (int): Maximum multiplier for ratios\n        \n    Returns:\n        pandas.DataFrame: Screening results\n    \"\"\"\n    results = []\n    \n    for ratio in target_ratios:\n        for mult in range(1, max_mult + 1):\n            scaled_ratio = [r * mult for r in ratio]\n            comp = Composition(dict(zip(elements, scaled_ratio))).reduced_composition\n            \n            is_valid = smact_validity(comp)\n            \n            results.append({\n                'elements': '-'.join(elements),\n                'target_ratio': ':'.join(map(str, ratio)),\n                'composition': comp,\n                'formula': comp.reduced_formula,\n                'valid': is_valid,\n                'structure_type': classify_structure_type(ratio)\n            })\n    \n    return pd.DataFrame(results).drop_duplicates('formula').reset_index(drop=True)\n\ndef classify_structure_type(ratio):\n    \"\"\"Classify structure type based on stoichiometric ratio.\"\"\"\n    if ratio == (1, 1, 3):\n        return \"Perovskite (ABX₃)\"\n    elif ratio == (1, 2, 4):\n        return \"Spinel (AB₂X₄)\"\n    elif ratio == (2, 1, 4):\n        return \"Inverse Spinel (A₂BX₄)\"\n    elif ratio == (1, 1, 4):\n        return \"Scheelite (ABX₄)\"\n    elif ratio == (1, 2, 6):\n        return \"Fluorite-related (AB₂X₆)\"\n    else:\n        return \"Other\"\n\n# Define interesting ternary systems and structure types\nternary_systems = [\n    ([\"Ca\", \"Al\", \"O\"], \"Calcium aluminate\"),\n    ([\"Li\", \"Co\", \"O\"], \"Battery cathode\"),\n    ([\"Y\", \"Ba\", \"O\"], \"Superconductor precursor\"),\n    ([\"Zn\", \"Al\", \"O\"], \"Transparent conductor\")\n]\n\n# Common structure type ratios to test\nstructure_ratios = [\n    (1, 1, 3),   # Perovskite\n    (1, 2, 4),   # Spinel\n    (2, 1, 4),   # Inverse spinel\n    (1, 1, 4),   # Scheelite\n    (1, 2, 6),   # Fluorite-related\n    (3, 1, 4),   # Other oxide\n    (1, 1, 2),   # Simple ternary\n]\n\nprint(\"Screening advanced ternary systems for multiple structure types...\\n\")\n\n# Create comprehensive results table\nall_ternary_results = []\n\nfor elements, description in ternary_systems:\n    print(f\"{description} ({'-'.join(elements)} system):\")\n    \n    # Screen all structure types for this system\n    results = screen_ternary_ratios(elements, structure_ratios, max_mult=2)\n    valid_results = results[results['valid']]\n    \n    print(f\"  Valid compositions: {len(valid_results)}/{len(results)}\")\n    \n    if len(valid_results) > 0:\n        print(\"  Valid structures found:\")\n        for _, row in valid_results.iterrows():\n            print(f\"    {row['formula']} - {row['structure_type']}\")\n    else:\n        print(\"    No valid compositions found\")\n    \n    # Add system description to results\n    results['system_description'] = description\n    all_ternary_results.append(results)\n    print()\n\n# Combine all results\ncombined_ternary_df = pd.concat(all_ternary_results, ignore_index=True)\n\n# Analyse structure type preferences\nstructure_analysis = combined_ternary_df[combined_ternary_df['valid']].groupby('structure_type').size().sort_values(ascending=False)\n\nprint(\"Structure type analysis (valid compositions):\")\nfor structure_type, count in structure_analysis.items():\n    print(f\"  {structure_type}: {count} valid systems\")\n\n# Create a summary visualisation\nplt.figure(figsize=(12, 8))\n\n# Create a matrix showing which systems support which structures\nsystems = [desc for _, desc in ternary_systems]\nstructures = structure_analysis.index.tolist()\n\nmatrix = np.zeros((len(systems), len(structures)))\n\nfor i, (elements, description) in enumerate(ternary_systems):\n    system_results = combined_ternary_df[\n        (combined_ternary_df['system_description'] == description) & \n        (combined_ternary_df['valid'])\n    ]\n    \n    for j, structure in enumerate(structures):\n        if structure in system_results['structure_type'].values:\n            matrix[i, j] = 1\n\n# Plot heatmap\nplt.imshow(matrix, cmap='RdYlGn', aspect='auto')\nplt.colorbar(label='Structure Type Supported')\nplt.xticks(range(len(structures)), structures, rotation=45, ha='right')\nplt.yticks(range(len(systems)), systems)\nplt.title('Ternary Structure Type Compatibility Matrix')\nplt.xlabel('Structure Type')\nplt.ylabel('Chemical System')\n\n# Add text annotations\nfor i in range(len(systems)):\n    for j in range(len(structures)):\n        text = 'Yes' if matrix[i, j] == 1 else 'No'\n        colour = 'black' if matrix[i, j] == 1 else 'red'\n        plt.text(j, i, text, ha='center', va='center', color=colour, weight='bold')\n\nplt.tight_layout()\nplt.show()\n\nprint(\"\\nKey insights:\")\nprint(\"- Different chemical systems favour different structure types\")\nprint(\"- Perovskite (ABX₃) structure is most commonly achievable\")\nprint(\"- Some systems support multiple structure types\")\nprint(\"- Battery and superconductor materials show distinct stoichiometric preferences\")"

In [ ]:
# Step 4: Advanced Stoichiometric Analysis with ICSD Data

print("Exploring ICSD oxidation state occurrence data...")

# Load ICSD oxidation state filter
icsd_filter = ICSD24OxStatesFilter()
occurrence_data = icsd_filter.get_species_occurrences_df(sort_by_occurrences=True)

print("\nMost common oxidation states in experimental structures:")
print(occurrence_data.head(10)[['species', 'results_count']])

# Analyse our perovskite results in context of ICSD data
def analyse_oxidation_states(formula):
    """
    Analyse the likely oxidation states for a given formula.
    This is a simplified analysis - in practice, you'd want to
    systematically test all valid oxidation state combinations.
    """
    comp = Composition(formula)
    
    analysis = []
    for element, amount in comp.items():
        element_obj = Element(element.symbol)
        common_states = element_obj.oxidation_states[:3]  # Most common states
        analysis.append(f"{element.symbol}: {common_states}")
    
    return "; ".join(analysis)

print("\nOxidation state analysis for valid perovskites:")
for _, row in valid_perovskites.iterrows():
    ox_analysis = analyse_oxidation_states(row['formula'])
    print(f"  {row['formula']}: {ox_analysis}")

# Plot ICSD occurrence distribution
plt.figure(figsize=(10, 6))
top_20 = occurrence_data.head(20)
plt.barh(range(len(top_20)), top_20['results_count'])
plt.yticks(range(len(top_20)), top_20['species'])
plt.xlabel('Number of occurrences in ICSD')
plt.title('Most Common Oxidation States in Experimental Structures')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nKey insight: O²⁻ is by far the most common oxidation state,")
print("appearing in over 116,000 structures. This validates our focus on oxides.")

In [ ]:
# Step 5: Comprehensive Stoichiometric Survey

def comprehensive_stoichiometric_survey(elements, max_stoich=3):
    """
    Perform a comprehensive survey of all possible stoichiometries
    for a given set of elements.
    
    Args:
        elements (list): Elements to survey
        max_stoich (int): Maximum stoichiometric coefficient
        
    Returns:
        pandas.DataFrame: Comprehensive results
    """
    results = []
    
    # Generate all possible stoichiometric combinations
    coefficients = range(1, max_stoich + 1)
    
    for stoich_combo in itertools.product(coefficients, repeat=len(elements)):
        # Create composition
        comp_dict = dict(zip(elements, stoich_combo))
        comp = Composition(comp_dict).reduced_composition
        
        # Test validity
        is_valid = smact_validity(comp)
        
        results.append({
            'elements': '-'.join(elements),
            'stoichiometry': ':'.join(map(str, stoich_combo)),
            'composition': comp,
            'formula': comp.reduced_formula,
            'valid': is_valid,
            'num_atoms': comp.num_atoms,
            'num_elements': len(comp.elements)
        })
    
    # Remove duplicates and sort
    df = pd.DataFrame(results)
    df = df.drop_duplicates('formula').reset_index(drop=True)
    df = df.sort_values(['valid', 'num_atoms'], ascending=[False, True])
    
    return df

# Example: Survey Cu-Ti-O system
print("Performing comprehensive stoichiometric survey of Cu-Ti-O system...")
cto_survey = comprehensive_stoichiometric_survey(["Cu", "Ti", "O"], max_stoich=3)

print(f"\nSurvey results summary:")
print(f"Total unique compositions: {len(cto_survey)}")
print(f"Valid compositions: {cto_survey['valid'].sum()}")
print(f"Success rate: {cto_survey['valid'].mean()*100:.1f}%")

# Show valid compositions
valid_cto = cto_survey[cto_survey['valid']]
print(f"\nValid Cu-Ti-O stoichiometries (ordered by complexity):")
for i, (_, row) in enumerate(valid_cto.head(8).iterrows(), 1):
    print(f"  {i:2d}. {row['formula']} ({row['num_atoms']} atoms)")

if len(valid_cto) > 8:
    print(f"     ... and {len(valid_cto)-8} more valid compositions")

In [ ]:
# Step 6: Interactive Ternary Visualisation

def create_ternary_stoichiometry_plot(df, elements):
    """
    Create an interactive ternary plot showing stoichiometric validity.
    
    Args:
        df (DataFrame): Survey results
        elements (list): Three elements for ternary plot
    """
    def composition_to_fractions(comp, element_list):
        """Convert composition to fractional coordinates."""
        amounts = [comp[el] for el in element_list]
        total = sum(amounts)
        return [amt/total for amt in amounts] if total > 0 else [0, 0, 0]
    
    # Convert compositions to ternary coordinates
    fractions = np.array([composition_to_fractions(row['composition'], elements) 
                         for _, row in df.iterrows()])
    
    # Create figure
    fig = go.Figure()
    
    # Plot valid compositions
    valid_mask = df['valid'].values
    if valid_mask.any():
        valid_fractions = fractions[valid_mask]
        valid_formulas = df[valid_mask]['formula'].values
        
        fig.add_trace(go.Scatterternary(
            a=valid_fractions[:, 0],
            b=valid_fractions[:, 1], 
            c=valid_fractions[:, 2],
            mode="markers",
            marker=dict(size=10, color="green", symbol="circle"),
            text=valid_formulas,
            name="Valid Stoichiometries",
            hovertemplate="<b>%{text}</b><br>" +
                         f"{elements[0]}: %{{a:.3f}}<br>" +
                         f"{elements[1]}: %{{b:.3f}}<br>" +
                         f"{elements[2]}: %{{c:.3f}}<extra></extra>"
        ))
    
    # Plot invalid compositions
    invalid_mask = ~valid_mask
    if invalid_mask.any():
        invalid_fractions = fractions[invalid_mask]
        invalid_formulas = df[invalid_mask]['formula'].values
        
        fig.add_trace(go.Scatterternary(
            a=invalid_fractions[:, 0],
            b=invalid_fractions[:, 1],
            c=invalid_fractions[:, 2],
            mode="markers",
            marker=dict(size=8, color="red", symbol="x"),
            text=invalid_formulas,
            name="Invalid Stoichiometries",
            hovertemplate="<b>%{text}</b><br>" +
                         f"{elements[0]}: %{{a:.3f}}<br>" +
                         f"{elements[1]}: %{{b:.3f}}<br>" +
                         f"{elements[2]}: %{{c:.3f}}<extra></extra>"
        ))
    
    # Update layout
    fig.update_layout(
        title=f"Stoichiometric Validity in {'-'.join(elements)} System",
        ternary=dict(
            aaxis=dict(title=elements[0], titlefont_size=14),
            baxis=dict(title=elements[1], titlefont_size=14),
            caxis=dict(title=elements[2], titlefont_size=14),
            bgcolor="rgba(0,0,0,0)"
        ),
        width=700,
        height=700,
        showlegend=True
    )
    
    fig.show()

# Create ternary plot for Cu-Ti-O system
create_ternary_stoichiometry_plot(cto_survey, ["Cu", "Ti", "O"])

print("\nTernary plot interpretation:")
print("- Green circles: Chemically valid stoichiometries")
print("- Red X marks: Invalid stoichiometries (violate chemical rules)")
print("- Position shows relative atomic fractions in ternary space")
print("- Hover over points to see exact compositions and formulas")

In [ ]:
# Summary: Mastering Stoichiometric Screening

print("Congratulations! You've completed stoichiometric screening training.")
print("=" * 65)

print("\nKEY CONCEPTS MASTERED:")
print("- Systematic generation of stoichiometric combinations")
print("- Chemical validation of atomic ratios")
print("- Binary, ternary, and multi-component screening")
print("- ICSD occurrence data for evidence-based screening")
print("- Interactive visualisation of stoichiometric spaces")

print("\nTHE SCREENING WORKFLOW:")
print("1. Define element combinations of interest")
print("2. Generate systematic stoichiometric variations")
print("3. Apply SMACT chemical validity rules")
print("4. Analyse results using occurrence statistics")
print("5. Visualise valid stoichiometric spaces")
print("6. Prioritise compositions for synthesis/computation")

if 'cto_survey' in locals():
    valid_count = cto_survey['valid'].sum()
    total_count = len(cto_survey)
    print(f"\nYOUR ANALYSIS RESULTS:")
    print(f"- Screened {total_count} stoichiometric combinations")
    print(f"- Found {valid_count} chemically valid compositions")
    print(f"- Achieved {valid_count/total_count*100:.1f}% success rate")

print("\nNEXT STEPS:")
print("- Apply stoichiometric screening to your research systems")
print("- Combine with computational property prediction")
print("- Use results to guide experimental synthesis plans")
print("- Integrate with high-throughput materials discovery workflows")

print("\nREMEMBER:")
print("Stoichiometric screening bridges the gap between compositional")
print("possibilities and practical synthesis targets. You now have the")
print("tools to systematically explore atomic ratio spaces with confidence!")

print("\n" + "=" * 65)
print("END OF STOICHIOMETRY SCREENING TUTORIAL")